In [1]:
!pip install tensorflow tensorflow-datasets flask flask-ngrok pillow


In [2]:
import os
os.makedirs("templates", exist_ok=True)
os.makedirs("static", exist_ok=True)


In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds

IMG_SIZE = 128

(ds_train, ds_test), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:20%]', 'train[80%:90%]'],  # small split for speed
    with_info=True,
    as_supervised=True,
)

def format_img(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32)/255.0
    return image, label

train = ds_train.map(format_img).batch(32).prefetch(1)
test = ds_test.map(format_img).batch(32).prefetch(1)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.1CAEC4_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.


In [4]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(train, epochs=10, validation_data=test)
model.save("dog_cat_model.h5")


Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


146/146 ━━━━━━━━━━━━━━━━━━━━ 231s 2s/step - accuracy: 0.5325 - loss: 0.6926 - val_accuracy: 0.6152 - val_loss: 0.6397
Epoch 2/10
146/146 ━━━━━━━━━━━━━━━━━━━━ 212s 1s/step - accuracy: 0.6505 - loss: 0.6292 - val_accuracy: 0.6681 - val_loss: 0.5987
Epoch 3/10
146/146 ━━━━━━━━━━━━━━━━━━━━ 193s 1s/step - accuracy: 0.6995 - loss: 0.5686 - val_accuracy: 0.6806 - val_loss: 0.5864
Epoch 4/10
146/146 ━━━━━━━━━━━━━━━━━━━━ 199s 1s/step - accuracy: 0.7507 - loss: 0.5061 - val_accuracy: 0.7438 - val_loss: 0.5279
Epoch 5/10
146/146 ━━━━━━━━━━━━━━━━━━━━ 203s 1s/step - accuracy: 0.7969 - loss: 0.4408 - val_accuracy: 0.7227 - val_loss: 0.5831
Epoch 6/10
146/146 ━━━━━━━━━━━━━━━━━━━━ 200s 1s/step - accuracy: 0.8377 - loss: 0.3521 - val_accuracy: 0.7429 - val_loss: 0.6102
Epoch 7/10
146/146 ━━━━━━━━━━━━━━━━━━━━ 188s 1s/step - accuracy: 0.8693 - loss: 0.2974 - val_accuracy: 0.7511 - val_loss: 0.6833
Epoch 8/10
146/146 ━━━━━━━━━━━━━━━━━━━━ 224s 2s/step - accuracy: 0.9008 - loss: 0.2347 - val_accuracy: 0.738

In [8]:
%%writefile app.py
from flask import Flask, render_template, request, jsonify
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from PIL import Image
import io
import base64

app = Flask(__name__)

# Load trained model
model = load_model("dog_cat_model.h5")

@app.route("/")
def index():
    return render_template("index.html")

@app.route("/predict", methods=["POST"])
def predict():
    try:
        file = request.files["file"]
        img = Image.open(file.stream).resize((128,128))
        img = np.array(img)/255.0
        img = np.expand_dims(img, axis=0)

        pred = model.predict(img)[0][0]
        label = "Dog 🐶" if pred > 0.5 else "Cat 🐱"
        return jsonify({"prediction": label, "confidence": float(pred)})
    except Exception as e:
        return jsonify({"error": str(e)}), 400

if __name__ == "__main__":
    app.run()


Overwriting app.py


In [9]:
html_content = """<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>Dog vs Cat Classifier</title>
<link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
<main class="container">
<h1>Dog 🐶 vs Cat 🐱 Classifier</h1>
<form id="uploadForm" class="upload-form" enctype="multipart/form-data">
<input type="file" id="fileInput" name="file" accept="image/*" required>
<button type="submit">Predict</button>
</form>
<section id="result" class="result"></section>
</main>

<script>
for(let i=0;i<30;i++){
  const spark=document.createElement('div');
  spark.className='sparks';
  spark.style.left=Math.random()*100+'%';
  spark.style.animationDuration=(2+Math.random()*3)+'s';
  spark.style.width=spark.style.height=(2+Math.random()*3)+'px';
  document.body.appendChild(spark);
}

const form=document.getElementById("uploadForm");
const resultDiv=document.getElementById("result");
form.addEventListener("submit", async (e)=>{
  e.preventDefault();
  const file=document.getElementById("fileInput").files[0];
  if(!file) return;
  const formData=new FormData();
  formData.append("file",file);
  try{
    const res=await fetch("/predict",{method:"POST",body:formData});
    const data=await res.json();
    if(data.error){
      resultDiv.innerHTML=`<p class="error">${data.error}</p>`;
    }else{
      resultDiv.innerHTML=`<h2>Prediction: ${data.prediction}</h2>
      <p>Confidence: ${(data.confidence*100).toFixed(2)}%</p>`;
    }
  }catch(err){
    resultDiv.innerHTML=`<p class="error">An error occurred. Try again.</p>`;
    console.error(err);
  }
});
</script>
</body>
</html>
"""

with open("templates/index.html", "w") as f:
    f.write(html_content)


In [10]:
css_content = """:root {
--primary-color:#ff7f50;
--secondary-color:#6a5acd;
--hover-color:#ff6347;
--text-color:#fff;
--bg-gradient:linear-gradient(135deg,#ff7f50,#6a5acd);
--font:'Arial',sans-serif;
}
*{box-sizing:border-box;margin:0;padding:0;}
body{font-family:var(--font);color:var(--text-color);background:#1a1a1a;
display:flex;justify-content:center;align-items:center;min-height:100vh;padding:20px;overflow:hidden;position:relative;}
@keyframes sparkle{0%{transform:translate(0,0) scale(0.5);opacity:1;}100%{transform:translate(200px,-400px) scale(1);opacity:0;}}
.sparks{position:absolute;width:4px;height:4px;background:#fff;border-radius:50%;top:50%;left:50%;animation:sparkle 3s linear infinite;}
.container{text-align:center;background:var(--bg-gradient);padding:40px 30px;border-radius:20px;
box-shadow:0 10px 25px rgba(0,0,0,0.5);width:100%;max-width:400px;position:relative;overflow:hidden;}
h1{margin-bottom:30px;font-size:2rem;text-shadow:2px 2px 5px rgba(0,0,0,0.5);}
.upload-form{display:flex;flex-direction:column;gap:15px;align-items:center;}
input[type="file"]{padding:10px;border-radius:8px;border:none;cursor:pointer;font-weight:bold;}
button{padding:12px 25px;background-color:var(--secondary-color);color:var(--text-color);border:none;
border-radius:12px;cursor:pointer;font-weight:bold;font-size:1rem;transition:all 0.3s ease;box-shadow:0 5px 15px rgba(0,0,0,0.3);}
button:hover{background-color:var(--hover-color);transform:scale(1.05);box-shadow:0 8px 20px rgba(0,0,0,0.5);}
.result{margin-top:25px;font-size:18px;text-shadow:1px 1px 2px rgba(0,0,0,0.3);}
.error{color:#ff4d4d;font-weight:bold;}
"""

with open("static/style.css","w") as f:
    f.write(css_content)


In [14]:
!pip install flask pyngrok tensorflow


In [15]:
from pyngrok import ngrok

# Add your ngrok authtoken
ngrok.set_auth_token("33F44sv2yA9UbESC2LBFYhfPkUA_2HRhwE2UzGxcvrx7HbFv4")

In [16]:
from pyngrok import ngrok
import app as myapp

# Kill old tunnels
ngrok.kill()

# Open tunnel
public_url = ngrok.connect(5000)
print("🌍 Public URL:", public_url)

myapp.app.run(host="0.0.0.0", port=5000)


🌍 Public URL: NgrokTunnel: "https://susana-demoded-delaine.ngrok-free.dev" -> "http://localhost:5000"
 * Serving Flask app 'app'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [26/Sep/2025 17:14:36] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Sep/2025 17:14:37] "GET /static/style.css HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step


INFO:werkzeug:127.0.0.1 - - [26/Sep/2025 17:14:47] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Sep/2025 17:14:58] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Sep/2025 17:15:13] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Sep/2025 17:15:19] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Sep/2025 17:15:20] "GET /static/style.css HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step


...

INFO:werkzeug:127.0.0.1 - - [26/Sep/2025 17:18:16] "POST /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step


INFO:werkzeug:127.0.0.1 - - [26/Sep/2025 17:18:16] "POST /predict HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step


INFO:werkzeug:127.0.0.1 - - [26/Sep/2025 17:18:29] "POST /predict HTTP/1.1" 200 -
